# 函数

### 函数中的参数值传递方式

1. call by value (值传递，传递到函数中的参数值是一份复制，在函数中修改参数值，在函数中修改参数值*不影响*原值原值)
2. call by reference (引用传递，传递到函数中的参数值是其本身值，在函数中修改参数值*会影响*原值)



In [2]:
def func1(x):
    x += 1
    print(f"x value in fun is: {x}")

x = 1
func1(x)
print(f"x value out of fun is: {x}")

x value in fun is: 2
x value out of fun is: 1


In [8]:
def func2(x):
    x[0] = 'c'
    print(f"x value in fun is: {x}")

x = ['a', 'b']
func2(x)
print(f"x value out of fun is: {x}")

x value in fun is: ['c', 'b']
x value out of fun is: ['c', 'b']


In [7]:
def func2(x):
    x = x + ('c',)
    print(f"x value in fun is: {x}")

x = ('a', 'b')
func2(x)
print(f"x value out of fun is: {x}")

x value in fun is: ('a', 'b', 'c')
x value out of fun is: ('a', 'b')


> Effbot (aka Fredrik Lundh) has described Python's variable passing style as call-by-object: http://effbot.org/zone/call-by-object.htm

> Objects are allocated on the heap and pointers to them can be passed around anywhere.


>    - When you make an assignment such as x = 1000, a dictionary entry is created that maps the string "x" in the current namespace to a pointer to the integer object containing one thousand.
>    - When you update "x" with x = 2000, a new integer object is created and the dictionary is updated to point at the new object. The old one thousand object is unchanged (and may or may not be alive depending on whether anything else refers to the object).
>    - When you do a new assignment such as y = x, a new dictionary entry "y" is created that points to the same object as the entry for "x".
>    - Objects like strings and integers are immutable. This simply means that there are no methods that can change the object after it has been created. For example, once the integer object one-thousand is created, it will never change. Math is done by creating new integer objects.
>    - Objects like lists are mutable. This means that the contents of the object can be changed by anything pointing to the object. For example, x = []; y = x; x.append(10); print y will print [10]. The empty list was created. Both "x" and "y" point to the same list. The append method mutates (updates) the list object (like adding a record to a database) and the result is visible to both "x" and "y" (just as a database update would be visible to every connection to that database).

> https://stackoverflow.com/questions/9696495/python-when-is-a-variable-passed-by-reference-and-when-by-value

简单而言
- 对于 immutable 的数据类型（int、float、str、tuple等），是值传递，因为不能被修改，只能创建一个副本
- 对于 mutable 的数据类型（list、dict等），是引用传递，因为可以修改，为了高效传递值，所以就通过引用传递了


### 默认参数

在Python中可以使用默认参数来简化函数调用，默认参数只能位于非默认参数之后

In [11]:
def pow(num, power=2):
    '''
    定义一个求幂的函数，默认是2次幂
    '''
    return num ** power

print(f"pow(2,3)={pow(2,3)}")
print(f"pow(2)={pow(2)}")

pow(2,3)=8
pow(2)=4


In [14]:
def append_to(element, to=[]):
    to.append(element)
    return to

my_list = append_to(12)
print(f"my_list: {my_list}")

my_other_list = append_to(42)
print(f"my_other_list: {my_other_list}")

my_list: [12]
my_other_list: [12, 42]


上面代码的运行结果是

```
my_list: [12]
my_other_list: [12, 42]
```

而不是预期的

```
my_list: [12]
my_other_list: [42]
```

这是因为Python的默认参数是函数定义的时候执行一次的，不是每次调用的时候都执行

> Python’s default arguments are evaluated once when the function is defined, not each time the function is called (like it is in say, Ruby). This means that if you use a mutable default argument and mutate it, you will and have mutated that object for all future calls to the function as well.
> https://docs.python-guide.org/writing/gotchas/

如果想实现预期效果，则代码应该是

```python
def append_to(element, to=None):
    if to is None:
        to = []
    to.append(element)
    return to
```

### 闭包

闭包简而言之——使得局部变量在函数外被访问成为可能

维基百科上的解释是:

> 在计算机科学中，闭包（Closure）是词法闭包（Lexical Closure）的简称，是引用了自由变量的函数。这个被引用的自由变量将和这个函数一同存在，即使已经离开了创造它的环境也不例外。所以，有另一种说法认为闭包是由函数和与其相关的引用环境组合而成的实体。

可以参考以下两篇文章

- https://foofish.net/python-closure.html
- http://www.ruanyifeng.com/blog/2009/08/learning_javascript_closures.html

In [16]:
def print_msg():
    # print_msg 是外围函数
    msg = "zen of python"
    def printer():
        # printer 是嵌套函数
        print(msg)
    return printer

another = print_msg()
# 输出 zen of python
another()

zen of python


**为什么要使用闭包**

闭包避免了使用全局变量，此外，闭包允许将函数与其所操作的某些数据（环境）关连起来。这一点与面向对象编程是非常类似的，在面对象编程中，对象允许我们将某些数据（对象的属性）与一个或者多个方法相关联。


In [17]:
def adder(x):
    def wrapper(y):
        return x + y
    return wrapper

adder5 = adder(5)
# 输出 15
adder5(10)
# 输出 11
adder5(6)

11

所有函数都有一个 __closure__属性，如果这个函数是一个闭包的话，那么它返回的是一个由 cell 对象 组成的元组对象。cell 对象的cell_contents 属性就是闭包中的自由变量。

In [19]:
print(adder.__closure__)
print(adder5.__closure__)
print(adder5.__closure__[0].cell_contents)

None
(<cell at 0x000001A8606886A8: int object at 0x00007FFA15A693C0>,)
5


我们再来看一个闭包的例子

In [20]:
def create_multipliers():
    return [lambda x : i * x for i in range(5)]

for multiplier in create_multipliers():
    print(multiplier(2))

8
8
8
8
8


我们预期想输出

```
0
2
4
6
8
```

而实际输出的却是：

```
8
8
8
8
8
```

**为什么呢**

> Python’s closures are late binding. This means that the values of variables used in closures are looked up at the time the inner function is called.

应为Python的闭包是延时绑定的，是在最后被调用的时候才计算的

可以使用下面代码来实现预期效果

```python
def create_multipliers():
    multipliers = []

    for i in range(5):
        def multiplier(x):
            return i * x
        multipliers.append(multiplier)

    return multipliers
```

> https://docs.python-guide.org/writing/gotchas/

也可以使用*立即执行函数*来实现

> IIFE（ 立即调用函数表达式）是一个在定义时就会立即执行的  JavaScript 函数。

>>```javascript
(function () {
     statements
})();
```

> 这是一个被称为 自执行匿名函数 的设计模式，主要包含两部分。第一部分是包围在 圆括号运算符 () 里的一个匿名函数，这个匿名函数拥有独立的词法作用域。这不仅避免了外界访问此 IIFE 中的变量，而且又不会污染全局作用域。

> https://developer.mozilla.org/zh-CN/docs/Glossary/立即执行函数表达式

```python
def create_multipliers():
    multipliers = []

    for i in range(5):
        def multiplier(num):
            def _inner(x):
                return num * x
            return _inner
        multipliers.append(multiplier(i))

    return multipliers
```

下面的代码是对上面的立即执行函数用lambda简写的形式

In [29]:
def create_multipliers():
    return [(lambda num: (lambda x: num * x))(i) for i in range(5)]

for multiplier in create_multipliers():
    print(multiplier(2))

0
2
4
6
8
